In [75]:
import numpy as np
m=5
n=5
tab=np.zeros([m,n],dtype=int)
obs=np.zeros([m,n],dtype=int)
obs[1:,0]=1
obs[0,3:n-2]=1
obs[2:3,3]=1
obs[0,0:5]=1
obs[2,4]=1
obs[4,1:3]=1
tab[obs==1]=-1

In [76]:
tab

array([[-1, -1, -1, -1, -1],
       [-1,  0,  0,  0,  0],
       [-1,  0,  0, -1, -1],
       [-1,  0,  0,  0,  0],
       [-1, -1, -1,  0,  0]])

In [59]:
constrain=[]
c=[[1,0],2,False,6] #[[i,j],n,(True is vertical ) or (False is herozintal),sum]
constrain.append(c)
c=[[2,3],2,True,5]
constrain.append(c)

In [60]:
def constrain_hope(table,constrain):
    if constrain==True:
        return True
    (i,j)=(constrain[0][0],constrain[0][1])
    n=constrain[1]
    sum=constrain[3]
    if constrain[2]:
        r=table[i+1:i+1+n,j]      
    else:
        r=table[i,j+1:j+1+n]
    if (np.unique(r[r!=0]).size)<(r[r!=0].size):
        return False
    if sum<np.sum(r):
        return False
    return True

In [61]:
def tab_solved(table,constrain):
   for c in constrain:
      (i,j)=(c[0][0],c[0][1])
      n=c[1]
      sum=c[3]
      if c[2]:
         r=table[i+1:i+1+n,j]      
      else:
         r=table[i,j+1:j+1+n]
      if not(sum==np.sum(r)):
         return False
      return True

In [62]:
def search_constrain_v(constrain,x,y):
    for i in constrain:
        if i[2]==True and i[0][1]==y and x>i[0][0] and x<=i[0][0]+i[1]:
            return i
    return True
def search_constrain_h(constrain,x,y):
    for i in constrain:
        if i[2]==False and i[0][0]==x and y>i[0][1] and y<=i[0][1]+i[1]:
            return i
    return True

In [64]:
def select_cell(tab):
    if not 0 in tab:
        return -1,-1
    else:
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                if tab[i,j]==0:
                    return i,j

In [72]:
def search(tab,constrain,selection=select_cell):
    r,c=selection(tab)
    if r==-1:
        if tab_solved(tab,constrain):
            return True
        else :
            return False
    c1=search_constrain_v(constrain,r,c)
    c2=search_constrain_h(constrain,r,c)
    for i in range(1,5):
        tab[r,c]=i
        if not(constrain_hope(tab,c1) and constrain_hope(tab,c2)):
            tab[r,c]=0
            continue
        if search(tab,constrain):
            return True
        tab[r,c]=0
    return False

In [73]:
search(tab,constrain)

True

In [68]:
tab

array([[-1, -1, -1, -1, -1],
       [-1,  2,  4,  1,  1],
       [-1,  1,  1, -1, -1],
       [-1,  1,  1,  1,  1],
       [-1, -1, -1,  2,  1]])

In [74]:
def select_cell_random(tab):
    if not 0 in tab:
        return -1,-1
    else:
        while True:
            a=np.random.randint(0,tab.shape[0],size=1)
            b=np.random.randint(0,tab.shape[1],size=1)
            if tab[a,b]==0:
                return a,b

In [77]:
search(tab,constrain,selection=select_cell_random)

True

In [79]:
tab

array([[-1, -1, -1, -1, -1],
       [-1,  2,  4,  1,  1],
       [-1,  1,  1, -1, -1],
       [-1,  1,  1,  2,  1],
       [-1, -1, -1,  1,  1]])

In [ ]:
def select_cell_intelligent(tab):
    if not 0 in tab:
        return -1,-1
    else:
        